# Logistic Regression

The most common way to do Bayesian Logistic Regression is to use the Laplace Approximation.

## The Model

The model is the same as Linear Regression, except the output is passed through a Logistic Sigmoid to give output that is on [0,1].

$P(\mathbf{w}) = \mathcal{N}(\mathbf{0}, \alpha^{-1}\mathbf{I})$

$P(\mathbf{y} \mid \mathbf{X}, \mathbf{w} ) = \sigma(\mathcal{N}(\mathbf{Xw}, \beta^{-1}\mathbf{I}))$

## No More Conjugacy

This model is not conjugate - what we typically do is is approximate the posterior with a Normal distribution. This is a useful approximation, as we then have a conjugate posterior-likelihood pair. This means if we want a predictive distribution, everything is easy.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(LogisticRegression, self).__init__()
        self.w = torch.nn.Linear(in_features, out_features)
        self.sigmoid = torch.nn.Sigmoid()
        self.bce = torch.nn.BCELoss()
        
    def forward(self, x):
        return self.sigmoid(self.w(x))
    
    def loss(self, x, y):
        y_hat = self.forward(x)
        l2 = torch.matmul(self.w.weight, self.w.weight.transpose(0,1)).sum()+self.w.bias**2
        return self.bce(y_hat, y) + l2
        

In [13]:
l = LogisticRegression(10,1)